In [1]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold,cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
os.chdir(r'C:\Users\dbda\Desktop\ML\Cases\Mohs_Dec23')

In [2]:
t_mohs=pd.read_csv("train.csv",index_col=0)
dum_t_mohs = pd.get_dummies(t_mohs, drop_first = True)

In [3]:
X = dum_t_mohs.drop('Hardness', axis = 1)

y = dum_t_mohs['Hardness']

### Ridge

In [4]:
from sklearn.linear_model import Ridge
ridge=Ridge()
params = {'alpha': np.linspace(0.001, 10,20)}
kfold = KFold(n_splits = 5, shuffle = True, random_state = 23)
gcv_rg= GridSearchCV(ridge, cv = kfold, param_grid = params)
gcv_rg.fit(X,y)
print('GCV best score: ', gcv_rg.best_score_)
print('GCV best parameters: ', gcv_rg.best_params_)

GCV best score:  0.24085489603846968
GCV best parameters:  {'alpha': 1.0535263157894736}


### Lasso

In [5]:
from sklearn.linear_model import Lasso
lasso=Lasso()
params = {'alpha': np.linspace(0.001, 10,20)}
kfold = KFold(n_splits = 5, shuffle = True, random_state = 23)
gcv = GridSearchCV(lasso, cv = kfold, param_grid = params)
gcv.fit(X,y)
print('GCV best score: ', gcv.best_score_)
print('GCV best parameters: ', gcv.best_params_)


GCV best score:  0.24069752131958483
GCV best parameters:  {'alpha': 0.001}


### Elastic Net

In [6]:
from sklearn.linear_model import ElasticNet
elastic=ElasticNet()
params = {'alpha': np.linspace(0.001, 5,20),
         'l1_ratio':np.linspace(0.001, 1,10)}
kfold = KFold(n_splits = 5, shuffle = True, random_state = 23)
gcv = GridSearchCV(elastic, cv = kfold, param_grid = params)
gcv.fit(X,y)
pd.DataFrame(gcv.cv_results_)
print('GCV best score: ', gcv.best_score_)
print('GCV best parameters: ', gcv.best_params_)

GCV best score:  0.24069752131958483
GCV best parameters:  {'alpha': 0.001, 'l1_ratio': 1.0}


### Best Estimator

In [7]:
best_rg=gcv_rg.best_estimator_

### Unlabelled data

In [12]:
tst=pd.read_csv(r"C:\Users\dbda\Desktop\ML\Cases\Mohs_Dec23\test.csv",index_col=0)


In [13]:
dum_tst=pd.get_dummies(tst,drop_first=True)
dum_tst.columns

Index(['allelectrons_Total', 'density_Total', 'allelectrons_Average',
       'val_e_Average', 'atomicweight_Average', 'ionenergy_Average',
       'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average',
       'zaratio_Average', 'density_Average'],
      dtype='object')

In [14]:
tst['Pred_Hardness']=best_rg.predict(dum_tst)

In [16]:
tst

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Pred_Hardness
id,,,,,,,,,,,,
10407,884.0,121.420000,35.360000,5.280000,82.561240,9.370384,2.298000,1.907200,1.200000,0.461899,1.79459,3.483739
10408,90.0,9.931960,18.000000,5.600000,39.568056,12.086300,2.868000,1.652000,0.864000,0.476196,1.41194,4.833068
10409,116.0,7.767992,11.600000,4.800000,23.231818,11.023840,2.644000,1.794000,0.960000,0.499514,0.78834,5.283257
10410,100.0,9.107996,10.000000,4.800000,20.298893,12.086300,2.828000,1.662000,0.792000,0.495796,1.20466,5.268807
10411,55.0,4.030000,11.000000,4.000000,22.977675,11.280950,2.455000,1.750000,0.893333,0.476095,0.93456,5.083274
...,...,...,...,...,...,...,...,...,...,...,...,...
17341,446.0,36.135392,9.636364,4.636364,19.689448,11.045515,2.768636,1.769697,0.871818,0.498702,0.11548,5.496252
17342,36.0,3.550000,18.000000,4.000000,36.068500,8.236600,1.790000,2.055000,1.390000,0.499000,1.35045,4.501663
17343,68.0,4.545328,11.333333,5.333333,22.688853,10.938358,2.876667,1.713333,0.916667,0.499074,0.73425,5.465102


In [36]:
prediction=best_rg.predict(dum_tst)
np.sum(prediction < 0)

2

In [38]:
prediction=np.where(prediction < 0,0,prediction)
np.sum(prediction < 0)

0

### Submission

In [39]:
submission=pd.read_csv(r"C:\Users\dbda\Desktop\ML\Cases\Mohs_Dec23\sample_submission.csv",index_col=False)

In [40]:
submission['Hardness']=prediction

In [41]:
submission

,id,Hardness
0,10407,3.483739
1,10408,4.833068
2,10409,5.283257
3,10410,5.268807
4,10411,5.083274
...,...,...
6934,17341,5.496252
6935,17342,4.501663
6936,17343,5.465102
6937,17344,4.368452


In [43]:
submission.to_csv("smt_mohs_2019.csv",index=False)